<a href="https://colab.research.google.com/github/Afreen89/-Detection-of-Pneumonia-in-Medical-Images/blob/main/ResNet50_Pneumonia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import keras,os
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D , Flatten
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
from tqdm import tqdm
import random
import cv2

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
! pip install tensorflow 
import tensorflow as tf

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
'''

def get_data(folder):
    X = []
    y = []
    for folderName in os.listdir(folder):
        if not folderName.startswith('.'):
            if folderName in ['no_pneumonia']:
                label = 0
            elif folderName in ['pneumonia']:
                label = 1
            else:
                label = 2
            for image_filename in tqdm(os.listdir(folder + folderName)):
                img_file = cv2.imread(folder + folderName + '/' + image_filename)
                if img_file is not None:
                    img_file = skimage.transform.resize(img_file, (150, 150, 3))
                    #img_file = scipy.misc.imresize(arr=img_file, size=(150, 150, 3))
                    img_arr = np.asarray(img_file)
                    X.append(img_arr)
                    y.append(label)
    X = np.asarray(X)
    y = np.asarray(y)
    return X,y

'''

In [5]:
CATEGORIES = ['no_pneumonia' , 'pneumonia']

In [6]:
train_dir = r'/content/drive/MyDrive/ML/Pneumonia/data-task1/train'
val_dir =  r'/content/drive/MyDrive/ML/Pneumonia/data-task1/val'
test_dir =  r'/content/drive/MyDrive/ML/Pneumonia/data-task1/test'
def pre_pro(DIRECTORY):

  data = []
  #pre-processing
  for category in CATEGORIES:
        folder = os.path.join(DIRECTORY, category)
        label = CATEGORIES.index(category)
        for img in tqdm(os.listdir(folder)):
                img_path = os.path.join(folder, img)
                #print(img_path)
                #img_arr = cv2.imread(img_path)
                #img_arr = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
                img_arr = cv2.imread(img_path)
                img_arr = cv2.resize(img_arr, (224, 224))
                data.append([img_arr, label])
           
  len(data)

  random.shuffle(data)

  X = []
  Y = []
    
  for features, labels in data:
   X.append(features)
   Y.append(labels)
    

  #X = np.array(X).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
  X = np.array(X)
  y = np.array(Y)
  return X, y
 


In [7]:
X_train, y_train = pre_pro(r'/content/drive/MyDrive/ML/Pneumonia/data-task1/train')
X_test, y_test= pre_pro(r'/content/drive/MyDrive/ML/Pneumonia/data-task1/test')
X_val, y_val= pre_pro(r'/content/drive/MyDrive/ML/Pneumonia/data-task1/val')


100%|██████████| 110/110 [00:03<00:00, 32.75it/s]


In [8]:
print(X_train.shape)
print(X_test.shape)
print(X_val.shape)

(3000, 224, 224, 3)
(500, 224, 224, 3)
(500, 224, 224, 3)


In [ ]:
'''
from keras.utils.np_utils import to_categorical
y_train = to_categorical(y_train, num_classes = 2)
y_test = to_categorical(y_test, num_classes = 2)
y_val = to_categorical(y_val, num_classes = 2)

In [9]:
print(y_train.shape)
print(y_test.shape)
print(y_val.shape)

(3000,)
(500,)
(500,)


In [10]:
# dont run,, pickle saved
import pickle
pickle.dump(X_train, open(r'/content/drive/MyDrive/ML/Pneumonia/data-task1/X_train.pkl', 'wb'))
pickle.dump(y_train, open(r'/content/drive/MyDrive/ML/Pneumonia/data-task1/y_train.pkl', 'wb'))
pickle.dump(X_test, open(r'/content/drive/MyDrive/ML/Pneumonia/data-task1/X_test.pkl', 'wb'))
pickle.dump(y_test, open(r'/content/drive/MyDrive/ML/Pneumonia/data-task1/y_test.pkl', 'wb'))
pickle.dump(X_val, open(r'/content/drive/MyDrive/ML/Pneumonia/data-task1/X_val.pkl', 'wb'))
pickle.dump(y_val, open(r'/content/drive/MyDrive/ML/Pneumonia/data-task1/y_val.pkl', 'wb'))



In [11]:
import pickle

X_train = pickle.load(open(r'/content/drive/MyDrive/ML/Pneumonia/data-task1/X_train.pkl','rb'))
y_train = pickle.load(open(r'/content/drive/MyDrive/ML/Pneumonia/data-task1/y_train.pkl','rb'))
X_test = pickle.load(open(r'/content/drive/MyDrive/ML/Pneumonia/data-task1/X_test.pkl','rb'))
y_test = pickle.load(open(r'/content/drive/MyDrive/ML/Pneumonia/data-task1/y_test.pkl','rb'))
X_val = pickle.load(open(r'/content/drive/MyDrive/ML/Pneumonia/data-task1/X_val.pkl','rb'))
y_val = pickle.load(open(r'/content/drive/MyDrive/ML/Pneumonia/data-task1/y_val.pkl','rb'))


In [12]:
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1. / 255)
val_datagen = ImageDataGenerator(rescale=1. / 255)


In [13]:
train_datagen.fit(X_train)
val_datagen.fit(X_val)
test_datagen.fit(X_test)

In [14]:
weight_for_0 = 0.35
weight_for_1 = 0.5

class_weight = {0:weight_for_0, 1: weight_for_1}


In [15]:
import seaborn as sns
import tensorflow as tf

In [16]:
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.layers import Dense, Activation, Conv2D, MaxPooling2D, Flatten, Dropout, BatchNormalization
from keras.models import load_model
from keras.models import Model
from tensorflow.keras import regularizers
ResNet50 = ResNet50(input_shape=[224,224,3], weights='imagenet', include_top=False)
for layer in ResNet50.layers:
    layer.trainable = False

X = ResNet50.output
X = BatchNormalization()(X)
X = Flatten()(X) 

X = Dense(512, activation='relu')(X) 
X = Dropout(0.5)(X)
#output = BatchNormalization()(X)

X = Dense(256, activation='relu')(X) 
X = Dropout(0.5)(X)
#output = BatchNormalization()(X)

X = Dense(128, activation='relu')(X) 
X = Dropout(0.5)(X)
#output = BatchNormalization()(X)

X = Dense(64, activation='relu')(X) 
X = Dropout(0.5)(X)
#output = BatchNormalization()(X)

output = Dense(2, activation='softmax')(X) 
model = Model(inputs=ResNet50.input, outputs=output)

94765736/94765736 [==============================] - 3s 0us/step


In [17]:
from tensorflow.keras.optimizers import Adam
opt = Adam(lr=0.03)

model.compile(optimizer='sgd', 
              loss=keras.losses.SparseCategoricalCrossentropy(), metrics=['accuracy'])

/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [18]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                              

In [19]:
from keras.callbacks import ModelCheckpoint, EarlyStopping
# '''
# checkpoint = ModelCheckpoint("vgg16_1.h5", monitor='val_acc', 
#                              verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)

# early = EarlyStopping(monitor='val_acc', 
#                       min_delta=0, patience=20, verbose=1, mode='auto')
# '''
hist = model.fit(X_train,y_train ,validation_data=(X_val,y_val), epochs=100, batch_size =32, class_weight= class_weight)

Epoch 1/100
94/94 [==============================] - 23s 136ms/step - loss: 0.3983 - accuracy: 0.6607 - val_loss: 0.4689 - val_accuracy: 0.7780
Epoch 2/100
94/94 [==============================] - 12s 126ms/step - loss: 0.2867 - accuracy: 0.7103 - val_loss: 0.4658 - val_accuracy: 0.8000
Epoch 3/100
94/94 [==============================] - 11s 115ms/step - loss: 0.2494 - accuracy: 0.7247 - val_loss: 0.4529 - val_accuracy: 0.7840
Epoch 4/100
94/94 [==============================] - 11s 116ms/step - loss: 0.2199 - accuracy: 0.7550 - val_loss: 0.4391 - val_accuracy: 0.7880
Epoch 5/100
94/94 [==============================] - 12s 130ms/step - loss: 0.1986 - accuracy: 0.7667 - val_loss: 0.4219 - val_accuracy: 0.8000
Epoch 6/100
94/94 [==============================] - 11s 119ms/step - loss: 0.1942 - accuracy: 0.7847 - val_loss: 0.4152 - val_accuracy: 0.8040
Epoch 7/100
94/94 [==============================] - 11s 120ms/step - loss: 0.1856 - accuracy: 0.7813 - val_loss: 0.4123 - val_accuracy:

In [20]:
from sklearn.metrics import confusion_matrix

# Visualization

In [21]:
model.save('path/ResNet50');

In [26]:
import matplotlib.pyplot as plt
plt.plot(hist.history["accuracy"])
plt.plot(hist.history["val_accuracy"])
plt.plot(hist.history["loss"])
plt.plot(hist.history["val_loss"])
plt.title("model accuracy - ResNet50")
plt.ylabel("Accuracy")
plt.xlabel("Epoch")
plt.legend(["Accuracy","Validation Accuracy","loss","Validation Loss"])
plt.show()

In [23]:
! pip install sklearn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for sklearn
  Running setup.py clean for sklearn
  error: subprocess-exited-with-error
  
  × python setup.py clean did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed cleaning build dir for sklearn
Failed to build sklearn
  error: subprocess-exited-with-error
  
  × Running setup.py install for sklearn did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Ru

In [ ]:
from tensorflow import keras
model = keras.models.load_model('path/to/location');

In [ ]:
import sklearn
from sklearn.metrics import confusion_matrix , classification_report
import numpy as np
import pandas as pd
from sklearn.metrics import precision_recall_fscore_support

pred = model.predict(X_test)

In [ ]:
pred 

In [ ]:
pred = np.argmax(pred, axis=1)

In [ ]:
y_true

In [ ]:

y_true = np.argmax(y_test,axis=1)

In [ ]:
pip install mlxtend

In [ ]:
import matplotlib.pyplot as plt

In [ ]:

CM = confusion_matrix(y_true,pred)
from mlxtend.plotting import plot_confusion_matrix
fig, ax = plot_confusion_matrix(conf_mat=CM, figsize=(5,5))
plt.show()

In [ ]:
y_test.shape

In [ ]:
from sklearn.metrics import accuracy_score
pred1 = model.predict(X_test)
accuracy = accuracy_score(y_test, np.round(pred1))*100

In [ ]:
pred1.shape

In [ ]:
print(CM)

In [ ]:
tn,fp,fn,tp = CM.ravel()

In [ ]:
prec = tp/(tp+fp)*100

In [ ]:
prec

In [ ]:
recall = tp/(tp+fn)*100
f1 = 2*prec*recall/(prec+recall)

In [ ]:
f1